# **Demo Script 2**


This script is the testing portion of the trained model analysis. Specifically this script loads in the test fNIRS and fMRI data, as well as the trained model (from demo 1). The trained model is then run on the testing data and the model performance is found.


In [1]:
import scipy.io as sio
import pickle

import numpy as np
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

## **Test the model on all of Run 2 Data**

In [2]:
## Test a model on each subject in Run 2

# Load fNIRS test data
fnirs_file2 = '../data/fnirs/clip2_noUncertain_zhbo_TxRxS.mat'
test_fNIRS = sio.loadmat(fnirs_file2)['zhbo_TxRxS'].transpose((2, 0, 1))  # reshape from TxRxS to SxTxR
# get shape
print(f'test_fNIRS shape is {test_fNIRS.shape}')

# let's impute missing channels with mean of all other channels
test_fNIRS = np.array(
    [
        np.apply_along_axis(lambda x: np.nan_to_num(x, nan=np.nanmean(x)), axis=1, arr=curr_region) \
            for curr_region in test_fNIRS.transpose((2, 1, 0))  # transpose to shape: RxTxS
    ]
).transpose((2, 1, 0))

# load fMRI test data
fmri_file2 = '../data/fmri/clip2_undenoised_withcartoon_fillmean_TxRxS.mat'
test_fMRI = sio.loadmat(fmri_file2)['bold_TxRxS'].transpose((2, 0, 1))  # reshape from TxRxS to SxTxR
test_fMRI_mean = np.mean(test_fMRI, axis=0)  # already imputed, so we use np.mean instead of np.nanmean
# get shape
print(f'test_fMRI shape is {test_fMRI.shape}')

# Load the model
file_path = '../models/sherlock_run1model.pickle'
with open(file_path, 'rb') as f:
   X_pca, y_pca, lr_reg = pickle.load(f)

# get pc loadings for X and y
X_pca_loadings = X_pca.components_
y_pca_loadings = y_pca.components_

print("Model loaded...")


test_fNIRS shape is (29, 1030, 20)
test_fMRI shape is (17, 1030, 122)
Model loaded...


/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.4.1.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.4.1.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
model_performance = []
model_predicted_tc = []
all_predicted_tc = []

for subj in range(test_fNIRS.shape[0]): # range(1): # range(test_fNIRS.shape[0]):
  curr_test_fNIRS = test_fNIRS[subj,:,:]
  fNIRS_test_pc = X_pca.transform(curr_test_fNIRS)
  model_predicted_PC = lr_reg.predict(X=fNIRS_test_pc)
  model_predicted_tc = y_pca.inverse_transform(model_predicted_PC)
  all_predicted_tc.append(model_predicted_tc)  # Append current prediction to the list
  
all_predicted_tc = np.stack(all_predicted_tc, axis=0)
mean_predicted_tc = np.mean(all_predicted_tc, axis=0)


NameError: name 'r_values' is not defined

In [9]:
r_values = np.zeros(test_fMRI_mean.shape[1])

for region in range(test_fMRI_mean.shape[1]):
    # Compute the Pearson correlation between the corresponding columns
    r_values[region], _ = pearsonr(mean_predicted_tc[:, region], test_fMRI_mean[:, region])


median_model_performance = np.median(r_values)

print(median_rvalues

[0.02370384 0.1304476  0.1101977  0.16218306 0.2206579  0.12373179
 0.05847062 0.03615254 0.03425395 0.10255468 0.11470048 0.1526089
 0.28526158 0.26305039 0.20785858 0.14195165 0.17029706 0.15562186
 0.09737807 0.22083148 0.24375524 0.12601387 0.15513758 0.16144531
 0.32132763 0.20078685 0.20937287 0.18286465 0.22880243 0.22934693
 0.20287114 0.36342864 0.32291313 0.34684867 0.44310676 0.2256804
 0.38955673 0.44837013 0.48477502 0.411051   0.42601698 0.3624241
 0.26507762 0.0675083  0.34680316 0.29863149 0.31536915 0.23862234
 0.53538248 0.46552648 0.48754915 0.4089595  0.36836853 0.23487166
 0.08108205 0.0471944  0.41901902 0.08448222 0.14807058 0.17846026
 0.21860491 0.2394337  0.0847428  0.0577899  0.02326    0.03549927
 0.0731244  0.15849488 0.15004488 0.30307511 0.18078733 0.23255673
 0.18815621 0.13094894 0.07224627 0.17368831 0.06135048 0.14273404
 0.26994531 0.0907178  0.14401546 0.16280381 0.44603788 0.38172941
 0.04458572 0.21022668 0.08674869 0.26778239 0.20908278 0.2144644

In [ ]:
#Saving the model data
# Create a dictionary to store the variables
data = {'all_predicted_tc': all_predicted_tc,
        'mean_predicted_tc': mean_predicted_tc,
        'median_model_performance': median_model_performance,
        'curr_test_fNIRS': curr_test_fNIRS,
        'fNIRS_test_pc': fNIRS_test_pc,
        'model_predicted_PC': model_predicted_PC,
        'model_performance': model_performance}

#Save the mat file
file_path = '../models/model_performance.mat'
sio.savemat(file_path, data)